Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/explain-model/explain-on-amlcompute/regression-sklearn-on-amlcompute.png)

# Train and explain models remotely via Azure Machine Learning Compute


_**This notebook showcases how to use the Azure Machine Learning Interpretability SDK to train and explain a regression model remotely on an Azure Machine Leanrning Compute Target (AMLCompute).**_




## Table of Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
    1. Initialize a Workspace
    1. Create an Experiment
    1. Introduction to AmlCompute
    1. Submit an AmlCompute run in a few different ways
        1. Option 1: Provision as a run based compute target 
        1. Option 2: Provision as a persistent compute target (Basic)
        1. Option 3: Provision as a persistent compute target (Advanced)
1. Additional operations to perform on AmlCompute
1. [Download model explanations from Azure Machine Learning Run History](#Download)
1. [Visualize explanations](#Visualize)
1. [Next steps](#Next)

## Introduction

This notebook showcases how to train and explain a regression model remotely via Azure Machine Learning Compute (AMLCompute), and download the calculated explanations locally for visualization.
It demonstrates the API calls that you need to make to submit a run for training and explaining a model to AMLCompute, download the compute explanations remotely, and visualizing the global and local explanations via a visualization dashboard that provides an interactive way of discovering patterns in model predictions and downloaded explanations.

We will showcase one of the tabular data explainers: TabularExplainer (SHAP).

Problem: Boston Housing Price Prediction with scikit-learn (train a model and run an explainer remotely via AMLCompute, and download and visualize the remotely-calculated explanations.)

| ![explanations-run-history](./img/explanations-run-history.PNG) |
|:--:|


## Setup
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration notebook](../../../configuration.ipynb) first if you haven't.


You will need to have extensions enabled prior to jupyter kernel starting to see the visualization dashboard.
```
(myenv) $ jupyter nbextension install --py --sys-prefix azureml.contrib.explain.model.visualize
(myenv) $ jupyter nbextension enable --py --sys-prefix azureml.contrib.explain.model.visualize
```
Or

```
(myenv) $ jupyter nbextension install azureml.contrib.explain.model.visualize --user --py
(myenv) $ jupyter nbextension enable azureml.contrib.explain.model.visualize --user --py
```

If you are using Jupyter Labs run the following commands instead:
```
(myenv) $ jupyter labextension install @jupyter-widgets/jupyterlab-manager
(myenv) $ jupyter labextension install microsoft-mli-widget
```

In [8]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.69


## Initialize a Workspace

Initialize a workspace object from persisted configuration

In [9]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

mersworkspace
myresourcegroup
eastus2
fac34303-435d-4486-8c3f-7094d82a0b60


## Create An Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [11]:
from azureml.core import Experiment
experiment_name = 'explainer-remote-run-tfworldCA'
experiment = Experiment(workspace=ws, name=experiment_name)

## Introduction to AmlCompute

Azure Machine Learning Compute is managed compute infrastructure that allows the user to easily create single to multi-node compute of the appropriate VM Family. It is created **within your workspace region** and is a resource that can be used by other users in your workspace. It autoscales by default to the max_nodes, when a job is submitted, and executes in a containerized environment packaging the dependencies as specified by the user. 

Since it is managed compute, job scheduling and cluster management are handled internally by Azure Machine Learning service. 

For more information on Azure Machine Learning Compute, please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)

If you are an existing BatchAI customer who is migrating to Azure Machine Learning, please read [this article](https://aka.ms/batchai-retirement)

**Note**: As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.


The training script `EmployeeAttritionClassification.py` is already created for you. Let's have a look.

## Submit an AmlCompute run in a few different ways

First lets check which VM families are available in your region. Azure is a regional service and some specialized SKUs (especially GPUs) are only available in certain regions. Since AmlCompute is created in the region of your workspace, we will use the supported_vms () function to see if the VM family we want to use ('STANDARD_D2_V2') is supported.

You can also pass a different region to check availability and then re-create your workspace in that region through the [configuration notebook](../../../configuration.ipynb)

In [12]:
from azureml.core.compute import ComputeTarget, AmlCompute

AmlCompute.supported_vmsizes(workspace=ws)
# AmlCompute.supported_vmsizes(workspace=ws, location='southcentralus')

[{'gpus': 0,
  'maxResourceVolumeMB': 51200,
  'memoryGB': 3.5,
  'name': 'Standard_D1_v2',
  'vCPUs': 1},
 {'gpus': 0,
  'maxResourceVolumeMB': 102400,
  'memoryGB': 7.0,
  'name': 'Standard_D2_v2',
  'vCPUs': 2},
 {'gpus': 0,
  'maxResourceVolumeMB': 204800,
  'memoryGB': 14.0,
  'name': 'Standard_D3_v2',
  'vCPUs': 4},
 {'gpus': 0,
  'maxResourceVolumeMB': 409600,
  'memoryGB': 28.0,
  'name': 'Standard_D4_v2',
  'vCPUs': 8},
 {'gpus': 0,
  'maxResourceVolumeMB': 102400,
  'memoryGB': 14.0,
  'name': 'Standard_D11_v2',
  'vCPUs': 2},
 {'gpus': 0,
  'maxResourceVolumeMB': 204800,
  'memoryGB': 28.0,
  'name': 'Standard_D12_v2',
  'vCPUs': 4},
 {'gpus': 0,
  'maxResourceVolumeMB': 409600,
  'memoryGB': 56.0,
  'name': 'Standard_D13_v2',
  'vCPUs': 8},
 {'gpus': 0,
  'maxResourceVolumeMB': 819200,
  'memoryGB': 112.0,
  'name': 'Standard_D14_v2',
  'vCPUs': 16},
 {'gpus': 0,
  'maxResourceVolumeMB': 7168,
  'memoryGB': 3.5,
  'name': 'Standard_DS1_v2',
  'vCPUs': 1},
 {'gpus': 0,
  'ma

### Create project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on

In [13]:
import os
import shutil

project_folder = './TFworld-explainer-remote-run-on-amlcompute'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train_explain-model-new.py', project_folder)

'./TFworld-explainer-remote-run-on-amlcompute/train_explain-model-new.py'

### Option 1: Provision as a run based compute target

You can provision AmlCompute as a compute target at run-time. In this case, the compute is auto-created for your run, scales up to max_nodes that you specify, and then **deleted automatically** after the run completes.

In [14]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# create a new runconfig object
run_config = RunConfiguration()

# signal that you want to use AmlCompute to execute script.
run_config.target = "amlcompute"

# AmlCompute will be created in the same region as workspace
# Set vm size for AmlCompute
run_config.amlcompute.vm_size = 'STANDARD_D2_V2'

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False


pip_packages = [
        'azureml-defaults', 'azureml-core', 'azureml-telemetry',
        'azureml-dataprep', 'sklearn', 'sklearn-pandas', 'tensorflow==1.14.0',
        'azureml-contrib-interpret', 'azureml-interpret'
]


index_url = 'https://azuremlsdktestpypi.azureedge.net/sdk-release/Candidate/604C89A437BA41BD942B4F46D9A3591D/'
run_config.environment.python.conda_dependencies = CondaDependencies.create(pip_packages=pip_packages,
                                                                            pin_sdk_version=False, 
                                                                            pip_indexurl=index_url)



# Now submit a run on AmlCompute
from azureml.core.script_run_config import ScriptRunConfig

script_run_config = ScriptRunConfig(source_directory=project_folder,
                                    script='train_explain-model-new.py',
                                    run_config=run_config)

run = experiment.submit(script_run_config)

# Show run details
run

Experiment,Id,Type,Status,Details Page,Docs Page
explainer-remote-run-tfworldCA,explainer-remote-run-tfworldCA_1572298501_3610a3a3,azureml.scriptrun,Starting,Link to Azure Portal,Link to Documentation


Note: if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

In [15]:
%%time
# Shows output of the run on stdout.
run.wait_for_completion(show_output=True)

RunId: explainer-remote-run-tfworldCA_1572298501_3610a3a3
Web View: https://mlworkspace.azure.ai/portal/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourceGroups/myresourcegroup/providers/Microsoft.MachineLearningServices/workspaces/mersworkspace/experiments/explainer-remote-run-tfworldCA/runs/explainer-remote-run-tfworldCA_1572298501_3610a3a3

Streaming azureml-logs/55_azureml-execution-tvmps_120dff54bfff86fa46b153cb27e284f2ae51d2267b5484038fdbe92dc1964266_d.txt

2019-10-28T21:39:38Z Starting output-watcher...
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_cacaf589266f385485375cfbb21695cb
a1298f4ce990: Pulling fs layer
04a3282d9c4b: Pulling fs layer
9b0d3db6dc03: Pulling fs layer
8269c605f3f1: Pulling fs layer
6504d449e70c: Pulling fs layer
4e38f320d0d4: Pulling fs layer
b0a763e8ee03: Pulling fs layer
11917a028ca4: Pulling fs layer
a6c378d11cbf: Pulling fs layer
6cc007ad9140: Pulling fs layer
6c1698a608f3: Pulling fs layer
6c141a7b86d3: Pulling 

2019-10-28 21:42:48.720899: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-10-28 21:42:48.725806: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394450000 Hz
2019-10-28 21:42:48.726031: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5f498a0 executing computations on platform Host. Devices:
2019-10-28 21:42:48.726144: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-10-28 21:42:48.760772: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or se

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "details": [],
        "message": "Could not locate the provided model_path outputs/model.h5 in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_120dff54bfff86fa46b153cb27e284f2ae51d2267b5484038fdbe92dc1964266_d.txt', 'azureml-logs/65_job_prep-tvmps_120dff54bfff86fa46b153cb27e284f2ae51d2267b5484038fdbe92dc1964266_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'explanation/fa67efec-fcb9-4c53-b435-89da28e99505/classes.interpret.json', 'explanation/fa67efec-fcb9-4c53-b435-89da28e99505/expected_values.interpret.json', 'explanation/fa67efec-fcb9-4c53-b435-89da28e99505/features.interpret.json', 'explanation/fa67efec-fcb9-4c53-b435-89da28e99505/global_importance_names/0.interpret.json', 'explanation/fa67efec-fcb9-4c53-b435-89da28e99505/global_importance_rank/0.interpret.json', 'explanation/fa67efec

## Download 
1. Download model explanation data.

In [8]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# Get model explanation data
client = ExplanationClient.from_run(run)
global_explanation = client.download_model_explanation()
local_importance_values = global_explanation.local_importance_values
expected_values = global_explanation.expected_values


/miniconda3/envs/MonInterpretDemo1/lib/python3.5/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Could not import lightgbm, required if using LGBMExplainableModel
Could not import lightgbm, required if using LGBMExplainableModel
Could not import lightgbm, required if using LGBMExplainableModel
Could not import lightgbm, required if using LGBMExplainableModel
Could not import lightgbm, required if using LGBMExplainableModel


In [9]:
# Or you can use the saved run.id to retrive the feature importance values
client = ExplanationClient.from_run_id(ws, experiment_name, run.id)
global_explanation = client.download_model_explanation()
local_importance_values = global_explanation.local_importance_values
expected_values = global_explanation.expected_values

In [10]:
# Get the top k (e.g., 4) most important features with their importance values
global_explanation_topk = client.download_model_explanation(top_k=4)
global_importance_values = global_explanation_topk.get_ranked_global_values()
global_importance_names = global_explanation_topk.get_ranked_global_names()

In [11]:
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

global importance values: [0.038821709294011034, 0.03154558415657708, 0.03120750383633114, 0.030372923769831986]
global importance names: ['StockOptionLevel', 'OverTime', 'EducationField', 'NumCompaniesWorked']


2. Download model file.

In [12]:
# retrieve model for visualization and deployment
from azureml.core.model import Model
from sklearn.externals import joblib
original_model = Model(ws, 'modelkeras')
model_path = original_model.download(exist_ok=True)


WebserviceException: WebserviceException:
	Message: ModelNotFound: Model with name modelkeras not found in provided workspace
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "ModelNotFound: Model with name modelkeras not found in provided workspace"
    }
}

In [ ]:
from tf.keras.models import load_model
model = load_model('model.h5')

3. Download test dataset.

In [ ]:
# retrieve x_test for visualization
from sklearn.externals import joblib
x_test_path = './x_test_boston_housing.pkl'
run.download_file('x_test_boston_housing.pkl', output_file_path=x_test_path)

In [ ]:
x_test = joblib.load('x_test_boston_housing.pkl')

## Visualize
Load the visualization dashboard

In [ ]:
from azureml.contrib.explain.model.visualize import ExplanationDashboard

In [ ]:
ExplanationDashboard(global_explanation, original_model, x_test)

## Next
Learn about other use cases of the explain package on a:
1. [Training time: regression problem](../../tabular-data/explain-binary-classification-local.ipynb)       
1. [Training time: binary classification problem](../../tabular-data/explain-binary-classification-local.ipynb)
1. [Training time: multiclass classification problem](../../tabular-data/explain-multiclass-classification-local.ipynb)
1. Explain models with engineered features:
    1. [Simple feature transformations](../../tabular-data/simple-feature-transformations-explain-local.ipynb)
    1. [Advanced feature transformations](../../tabular-data/advanced-feature-transformations-explain-local.ipynb)
1. [Save model explanations via Azure Machine Learning Run History](../run-history/save-retrieve-explanations-run-history.ipynb)
1. Inferencing time: deploy a classification model and explainer:
    1. [Deploy a locally-trained model and explainer](../scoring-time/train-explain-model-locally-and-deploy.ipynb)
    1. [Deploy a remotely-trained model and explainer](../scoring-time/train-explain-model-on-amlcompute-and-deploy.ipynb)